# Training an activity classifier

This notebook was used to train the activity classification model **GestureClassifier.mlmodel** supplied with the book Machine Learning by Tutorials.

In [ ]:
'''
/// Copyright (c) 2018 Razeware LLC
/// 
/// Permission is hereby granted, free of charge, to any person obtaining a copy
/// of this software and associated documentation files (the "Software"), to deal
/// in the Software without restriction, including without limitation the rights
/// to use, copy, modify, merge, publish, distribute, sublicense, and/or sell
/// copies of the Software, and to permit persons to whom the Software is
/// furnished to do so, subject to the following conditions:
/// 
/// The above copyright notice and this permission notice shall be included in
/// all copies or substantial portions of the Software.
/// 
/// Notwithstanding the foregoing, you may not use, copy, modify, merge, publish,
/// distribute, sublicense, create a derivative work, and/or sell copies of the
/// Software in any work that is designed, intended, or marketed for pedagogical or
/// instructional purposes related to programming, coding, application development,
/// or information technology.  Permission for such use, copying, modification,
/// merger, publication, distribution, sublicensing, creation of derivative works,
/// or sale is expressly withheld.
/// 
/// THE SOFTWARE IS PROVIDED "AS IS", WITHOUT WARRANTY OF ANY KIND, EXPRESS OR
/// IMPLIED, INCLUDING BUT NOT LIMITED TO THE WARRANTIES OF MERCHANTABILITY,
/// FITNESS FOR A PARTICULAR PURPOSE AND NONINFRINGEMENT. IN NO EVENT SHALL THE
/// AUTHORS OR COPYRIGHT HOLDERS BE LIABLE FOR ANY CLAIM, DAMAGES OR OTHER
/// LIABILITY, WHETHER IN AN ACTION OF CONTRACT, TORT OR OTHERWISE, ARISING FROM,
/// OUT OF OR IN CONNECTION WITH THE SOFTWARE OR THE USE OR OTHER DEALINGS IN
/// THE SOFTWARE.
'''

In [1]:
import turicreate as tc
import activity_detector_utils as utils

# Load clean datasets

In [2]:
train_sf = tc.SFrame("data/cleaned_train_sframe")
valid_sf = tc.SFrame("data/cleaned_valid_sframe")
test_sf = tc.SFrame("data/cleaned_test_sframe")

### Optional: Split a validation set from your training set

This step is **not** necessary because we already have a separate validation set. This project does not use the results of the following cell, but it's here as an example to show you how to do it *if* you ever want to in your own projects.

In [3]:
train, valid = tc.activity_classifier.util.random_split_by_session(train_sf, session_id='sessionId', fraction=0.9)

In [4]:
utils.count_activities(train)
utils.count_activities(valid)

+----------+--------+-------+
| activity | userId | Count |
+----------+--------+-------+
| chop_it  |  u_01  |   33  |
| chop_it  |  u_02  |   34  |
| drive_it |  u_01  |   31  |
| drive_it |  u_02  |   30  |
| rest_it  |  u_01  |   95  |
| rest_it  |  u_02  |   97  |
| shake_it |  u_01  |   31  |
| shake_it |  u_02  |   33  |
+----------+--------+-------+
[8 rows x 3 columns]

+----------+--------+-------+
| activity | userId | Count |
+----------+--------+-------+
| chop_it  |  u_01  |   3   |
| chop_it  |  u_02  |   2   |
| drive_it |  u_01  |   5   |
| drive_it |  u_02  |   6   |
| rest_it  |  u_01  |   13  |
| rest_it  |  u_02  |   11  |
| shake_it |  u_01  |   5   |
| shake_it |  u_02  |   3   |
+----------+--------+-------+
[8 rows x 3 columns]



# Verify your model correctness by overfitting

This step is to ensure the dataset and model are applicable to the problem, and the model is implemented properly. The following cell grabs a random 5% of the training data and trains an activity classifier model with it to ensure it learns successfully. You can see the training accuracy is quite high – at times 100% – which is a good indication that the model should be able to learn from your dataset.

In [5]:
tiny_train, _ = tc.activity_classifier.util.random_split_by_session(train_sf, session_id='sessionId', fraction=0.05)

tc.activity_classifier.create(
    dataset=tiny_train, session_id='sessionId', target='activity', 
    features=["rotX", "rotY", "rotZ", "accelX", "accelY", "accelZ"],
    prediction_window=20, max_iterations=50)

The dataset has less than the minimum of 100 sessions required for train-validation split. Continuing without validation set


Pre-processing 7096 samples...

Using sequences of size 400 for model creation.

Processed a total of 7 sessions.

Using GPU to create model (AMD Radeon R9 M295X)
+----------------+----------------+----------------+----------------+
| Iteration      | Train Accuracy | Train Loss     | Elapsed Time   |
+----------------+----------------+----------------+----------------+
| 1              | 0.295          | 1.626          | 0.0            |
| 2              | 0.580          | 1.068          | 0.1            |
| 3              | 0.674          | 0.874          | 0.1            |
| 4              | 0.807          | 0.653          | 0.1            |
| 5              | 0.861          | 0.502          | 0.2            |
| 6              | 0.901          | 0.419          | 0.2            |
| 7              | 0.904          | 0.433          | 0.2            |
| 8              | 0.921          | 0.374          | 0.3            |
| 9              | 0.914          | 0.351          | 0.3            |
| 10             | 0.929          | 0.276          | 0.3            |
| 11             | 0.935          | 0.268 

Class                                    : ActivityClassifier

Schema
------
Number of examples                       : 7096
Number of sessions                       : 7
Number of classes                        : 4
Number of feature columns                : 6
Prediction window                        : 20

Training summary
----------------
Log-likelihood                           : 0.0371
Training time (sec)                      : 1.7914

# Train the model

You'll get different results every time you run this training cell, because there's some randomness built into the training process. You may get slightly better or worse results, but they should be pretty close to these.

In [6]:
# Create an activity classifier
model = tc.activity_classifier.create(
    dataset=train_sf, session_id='sessionId', target='activity', 
    features=["rotX", "rotY", "rotZ", "accelX", "accelY", "accelZ"],
    prediction_window=20, validation_set=valid_sf, max_iterations=50)

Pre-processing 235057 samples...

Using sequences of size 400 for model creation.

Processed a total of 216 sessions.

Using GPU to create model (AMD Radeon R9 M295X)
+---------------------+---------------------+---------------------+---------------------+---------------------+---------------------+
| Iteration           | Train Accuracy      | Train Loss          | Validation Accuracy | Validation Loss     | Elapsed Time        |
+---------------------+---------------------+---------------------+---------------------+---------------------+---------------------+
| 1                   | 0.562               | 1.209               | 0.713               | 0.705               | 0.6                 | 
| 2                   | 0.785               | 0.556               | 0.932               | 0.240               | 1.2                 | 
| 3                   | 0.882               | 0.349               | 0.952               | 0.153               | 1.8                 | 
| 4                   | 0.924               | 0.248               | 0.955               | 0.134               | 2.4                 | 
| 5       

In [7]:
model.summary()

Class                                    : ActivityClassifier

Schema
------
Number of examples                       : 235057
Number of sessions                       : 216
Number of classes                        : 4
Number of feature columns                : 6
Prediction window                        : 20

Training summary
----------------
Log-likelihood                           : 0.0434
Training time (sec)                      : 30.0827



## Evaluate trained model

In [8]:
metrics = model.evaluate(test_sf)

In [9]:
print(f"Accuracy: {metrics['accuracy']}")

Accuracy: 0.9734649489710557


In [10]:
print(metrics)

{'accuracy': 0.9734649489710557, 'auc': 0.9966916821874753, 'precision': 0.976516169734427, 'recall': 0.9682026404974962, 'f1_score': 0.9720395581549871, 'log_loss': 0.12406052127540747, 'confusion_matrix': Columns:
	target_label	str
	predicted_label	str
	count	int

Rows: 11

Data:
+--------------+-----------------+-------+
| target_label | predicted_label | count |
+--------------+-----------------+-------+
|   shake_it   |     rest_it     |   96  |
|   rest_it    |     drive_it    |   17  |
|   drive_it   |     drive_it    |  5443 |
|   chop_it    |     rest_it     |  181  |
|   chop_it    |     shake_it    |  176  |
|   rest_it    |     rest_it     | 13164 |
|   chop_it    |     chop_it     |  5118 |
|   drive_it   |     rest_it     |  184  |
|   shake_it   |     shake_it    |  5367 |
|   rest_it    |     chop_it     |   62  |
+--------------+-----------------+-------+
[11 rows x 3 columns]
Note: Only the head of the SFrame is printed.
You can use print_rows(num_rows=m, num_columns=

In [11]:
metrics['confusion_matrix'].print_rows(num_rows=11)

+--------------+-----------------+-------+
| target_label | predicted_label | count |
+--------------+-----------------+-------+
|   shake_it   |     rest_it     |   96  |
|   rest_it    |     drive_it    |   17  |
|   drive_it   |     drive_it    |  5443 |
|   chop_it    |     rest_it     |  181  |
|   chop_it    |     shake_it    |  176  |
|   rest_it    |     rest_it     | 13164 |
|   chop_it    |     chop_it     |  5118 |
|   drive_it   |     rest_it     |  184  |
|   shake_it   |     shake_it    |  5367 |
|   rest_it    |     chop_it     |   62  |
|   rest_it    |     shake_it    |   77  |
+--------------+-----------------+-------+
[11 rows x 3 columns]



## Save the trained model

Export the model to Core ML and save a copy you can reload here if you want to do anything else with it (e.g. test it on a different dataset)

In [12]:
model.export_coreml("GestureClassifier.mlmodel")
model.save("GestureClassifier")